In [1]:
%autosave 0
# %matplotlib inline
%matplotlib notebook

from __future__ import division, print_function
import numpy as np
from tensorflow.python.client import device_lib
from keras import backend as K
import tensorflow as tf
import keras
from keras.datasets import cifar10


print(device_lib.list_local_devices())
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(keras.__version__)

Autosave disabled
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 707702170077368593
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5132910592
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 4376826745162654827
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:06:00.0, compute capability: 3.7"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 6853368218
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 16500082982957713437
physical_device_desc: "device: 1, name: Tesla K80, pci bus id: 0000:07:00.0, compute capability: 3.7"
]
2.1.6


Using TensorFlow backend.


In [2]:
from keras.layers import Input, Dense, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Add, Activation, Conv2DTranspose, Reshape
from keras.models import Model, load_model
from keras import optimizers
# from keras.utils import multi_gpu_model

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
input_train_shape = x_train.shape[1:]

n_out = 10
n_fc1 = 256

rc_model = load_model('cifar_recurrentcomplex_180712.h5');
conv1_ffw = rc_model.get_layer('conv_ff1')
conv1_rcw = rc_model.get_layer('conv_rc1')
conv2_ffw = rc_model.get_layer('conv_ff2')
conv2_rcw = rc_model.get_layer('conv_rc2')
conv3_ffw = rc_model.get_layer('conv_ff3')
conv3_rcw = rc_model.get_layer('conv_rc3')
conv4_ffw = rc_model.get_layer('conv_ff4')
conv4_rcw = rc_model.get_layer('conv_rc4')
fc5_ffw = rc_model.get_layer('fc_ff5')
fc5_rcw = rc_model.get_layer('fc_rc5')
fc6_ffw = rc_model.get_layer('fc_ff6')
fc6_rcw = rc_model.get_layer('fc_rc6')
fc7_ffw = rc_model.get_layer('fc_ff7')
fc7_rcw = rc_model.get_layer('fc_rc7')

input_layer = Input(shape=(input_train_shape), name='input')

#Layer 1 complex
conv1_ff1 = Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_ff11', trainable=False, weights=conv1_ffw.get_weights())(input_layer)
conv1_rc1 = Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_rc11', trainable=False, weights=conv1_rcw.get_weights())(conv1_ff1)
conv1_mrg1 = Add()([conv1_ff1, conv1_rc1])
conv1_rc2 = Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_rc12', trainable=False, weights=conv1_rcw.get_weights())(conv1_mrg1)
bn11 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_11')(conv1_mrg1)
pool11 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool_11')(bn11)
conv2_ff1 = Conv2D(128, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_ff21', trainable=False, weights=conv2_ffw.get_weights())(pool11)
deconv1_td1 = Conv2DTranspose(64, (2,2), strides=(2,2), padding='valid', activation='relu', name='deconv_td1', trainable=True)(conv2_ff1)
conv1_mrg2 = Add()([conv1_ff1, conv1_rc2, deconv1_td1])
bn12 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_12')(conv1_mrg2)
pool12 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool_12')(bn12)

#Layer 2 complex
conv2_ff2 = Conv2D(128, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_ff22', trainable=False, weights=conv2_ffw.get_weights())(pool12)
conv2_rc1 = Conv2D(128, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_rc21', trainable=False, weights=conv2_rcw.get_weights())(conv2_ff2)
conv2_mrg1 = Add()([conv2_ff2, conv2_rc1])
conv2_rc2 = Conv2D(128, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_rc22', trainable=False, weights=conv2_rcw.get_weights())(conv2_mrg1)
bn21 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_21')(conv2_mrg1)
pool21 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool_21')(bn21)
conv3_ff1 = Conv2D(256, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_ff31', trainable=False, weights=conv3_ffw.get_weights())(pool21)
deconv2_td1 = Conv2DTranspose(128, (2,2), strides=(2,2), padding='valid', activation='relu', name='deconv_td2', trainable=True)(conv3_ff1)
conv2_mrg2 = Add()([conv2_ff2, conv2_rc2, deconv2_td1])
bn22 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_22')(conv2_mrg2)
pool22 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool_22')(bn22)


# Layer 3 complex
conv3_ff2 = Conv2D(256, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_ff32', trainable=False, weights=conv3_ffw.get_weights())(pool22)
conv3_rc1 = Conv2D(256, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_rc31', trainable=False, weights=conv3_rcw.get_weights())(conv3_ff2)
conv3_mrg1 = Add()([conv3_ff2, conv3_rc1])
conv3_rc2 = Conv2D(256, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_rc32', trainable=False, weights=conv3_rcw.get_weights())(conv3_mrg1)
bn31 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_31')(conv3_mrg1)
pool31 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool31')(bn31)
conv4_ff1 = Conv2D(1024, (4,4), strides=(1,1), padding='valid', activation='relu', name='conv_ff41', trainable=False, weights=conv4_ffw.get_weights())(pool31)
deconv3_td1 = Conv2DTranspose(256, (8,8), strides=(1,1), padding='valid', activation='relu', name='deconv_td3', trainable=True)(conv4_ff1)
conv3_mrg2 = Add()([conv3_ff2, conv3_rc2, deconv3_td1])
bn32 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_32')(conv3_mrg2)
pool32 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool_32')(bn32)

# Layer 4 complex
conv4_ff2 = Conv2D(1024, (4,4), strides=(1,1), padding='valid', activation='relu', name='conv_ff42', trainable=False, weights=conv4_ffw.get_weights())(pool32)
conv4_rc1 = Conv2D(1024, (1,1), strides=(1,1), padding='valid', activation='relu', name='conv_rc41', trainable=False, weights=conv4_rcw.get_weights())(conv4_ff2)
conv4_mrg1 = Add()([conv4_ff2, conv4_rc1])
conv4_rc2 = Conv2D(1024, (1,1), strides=(1,1), padding='valid', activation='relu', name='conv_rc42', trainable=False, weights=conv4_rcw.get_weights())(conv4_mrg1)
bn41 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_41')(conv4_mrg1)
flat11 = Flatten(name='flat_11')(bn41)
fc5_ff1 = Dense(1024, activation='relu', name='fc_ff51', trainable=False, weights=fc5_ffw.get_weights())(flat11)
deconv4_td1 = Dense(1024, activation='relu', name='deconv_td4', trainable=True)(fc5_ff1)
resh11 = Reshape((1,1,1024))(deconv4_td1)
conv4_mrg2 = Add()([conv4_ff2, conv4_rc2, resh11])
bn42 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_42')(conv4_mrg2)
flat12 = Flatten(name='flat_12')(bn42)


# Layer 5 complex
fc5_ff2 = Dense(1024, activation='relu', name='fc_ff52', trainable=False, weights=fc5_ffw.get_weights())(flat12)
fc5_rc1 = Dense(1024, activation='relu', name='fc_rc51', trainable=False, weights=fc5_rcw.get_weights())(fc5_ff2)
fc5_mrg1 = Add()([fc5_ff2, fc5_rc1])
fc5_rc2 = Dense(1024, activation='relu', name='fc_rc52', trainable=False, weights=fc5_rcw.get_weights())(fc5_mrg1)
bn51 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_51')(fc5_mrg1)
fc6_ff1 = Dense(256, activation='relu', name='fc_ff61', trainable=False, weights=fc6_ffw.get_weights())(bn51)
fc5_td1 = Dense(1024, activation='relu', name='fc_td5', trainable=True)(fc6_ff1)
fc5_mrg2 = Add()([fc5_ff2, fc5_rc2, fc5_td1])
bn52 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_52')(fc5_mrg2)

# Layer 6 complex
fc6_ff2 = Dense(256, activation='relu', name='fc_ff62', trainable=False, weights=fc6_ffw.get_weights())(bn52)
fc6_rc1 = Dense(256, activation='relu', name='fc_rc61', trainable=False, weights=fc6_rcw.get_weights())(fc6_ff2)
fc6_mrg1 = Add()([fc6_ff2, fc6_rc1])
fc6_rc2 = Dense(256, activation='relu', name='fc_rc62', trainable=False, weights=fc6_rcw.get_weights())(fc6_mrg1)
bn61 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_61')(fc6_mrg1)
fc7_ff1 = Dense(10, activation='linear', name='fc_ff71', trainable=False, weights=fc7_ffw.get_weights())(bn61)
fc6_td1 = Dense(256, activation='relu', name='fc_td6', trainable=True)(fc7_ff1)
fc6_mrg2 = Add()([fc6_ff2, fc6_rc2, fc6_td1])
bn62 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_62')(fc6_mrg2)

# Layer 7 complex
fc7_ff2 = Dense(10, activation='linear', name='fc_ff72', trainable=False, weights=fc7_ffw.get_weights())(bn62)
fc7_rc1 = Dense(10, activation='linear', name='fc_rc71', trainable=False, weights=fc7_rcw.get_weights())(fc7_ff2)
fc7_mrg1 = Add()([fc7_ff2, fc7_rc1])
output_layer = Activation('softmax')(fc7_mrg1)




/home/python/2.7/envs/tf1.8/lib/python2.7/site-packages/keras/models.py:318: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [3]:
sgd = optimizers.SGD(lr=2, decay=1e-6, momentum=0.9)
model = Model(input_layer, output_layer)
# pmodel = multi_gpu_model(model)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv_ff11 (Conv2D)              (None, 32, 32, 64)   1792        input[0][0]                      
__________________________________________________________________________________________________
conv_rc11 (Conv2D)              (None, 32, 32, 64)   36928       conv_ff11[0][0]                  
__________________________________________________________________________________________________
add_1 (Add)                     (None, 32, 32, 64)   0           conv_ff11[0][0]                  
                                                                 conv_rc11[0][0]                  
__________

In [4]:
print (x_train.shape)
print (y_train.shape)

# scale intensities between 0 and 1
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print('min:', x_train[0].min())
print('max:', x_train[0].max())


(50000, 32, 32, 3)
(50000, 1)
min: 0.0
max: 1.0


In [5]:
n_epochs = 5
n_batch_size = 4
labels_train = keras.utils.to_categorical(y_train, n_out)
labels_test = keras.utils.to_categorical(y_test, n_out)


history = model.fit(x_train, labels_train,
                    epochs=n_epochs,
                    batch_size=n_batch_size)

evaluation = model.evaluate(x_test, labels_test, verbose=0)

print('\n[Train parameters]')
for item in history.params:
    print(item+':', history.params[item])
    
print('\n[Model evaluation]')
print('test', history.params['metrics'][0], format(np.mean(evaluation[0]), '.4f'))
print('test', history.params['metrics'][1], format(np.mean(evaluation[1]), '.4f'))

Epoch 1/5
50000/50000 [==============================] - 1388s 28ms/step - loss: 1.1980 - acc: 0.5961
Epoch 2/5
50000/50000 [==============================] - 1330s 27ms/step - loss: 0.9384 - acc: 0.6847
Epoch 3/5
50000/50000 [==============================] - 1330s 27ms/step - loss: 0.8900 - acc: 0.7009
Epoch 4/5
50000/50000 [==============================] - 1331s 27ms/step - loss: 0.8575 - acc: 0.7133
Epoch 5/5
50000/50000 [==============================] - 1345s 27ms/step - loss: 0.8299 - acc: 0.7208

[Train parameters]
metrics: ['loss', 'acc']
samples: 50000
batch_size: 4
epochs: 5
steps: None
do_validation: False
verbose: 1

[Model evaluation]
test loss 1.0626
test acc 0.6970


In [6]:
model.save('cifar_rectopdowncomplex_180716.h5')

In [7]:
# Playground
conv1_ffw.set_weights(1)

TypeError: object of type 'int' has no len()